In [ ]:
# Import all league API modules
from scrapernhl.pwhl import api as pwhl_api
from scrapernhl.ahl import api as ahl_api
from scrapernhl.ohl import api as ohl_api
from scrapernhl.whl import api as whl_api
from scrapernhl.qmjhl import api as qmjhl_api

import pandas as pd
import json

## 1. Basic API Calls - Get Teams

Each league has the same `get_teams()` function that returns all teams for a season.

In [ ]:
# Get teams for each league
print("="*60)
print("TEAMS BY LEAGUE")
print("="*60)

leagues = [
    ("PWHL", pwhl_api),
    ("AHL", ahl_api),
    ("OHL", ohl_api),
    ("WHL", whl_api),
    ("QMJHL", qmjhl_api)
]

for league_name, api_module in leagues:
    teams = api_module.get_teams()
    
    # Extract team list (format may vary slightly)
    if isinstance(teams, dict):
        team_list = teams.get('teams', teams.get('SiteKit', {}).get('Team', []))
        if isinstance(team_list, list):
            print(f"\n{league_name}: {len(team_list)} teams")
            # Show first 3 teams as examples
            for team in team_list[:3]:
                if isinstance(team, dict):
                    print(f"  - {team.get('name', 'Unknown')}")

## 2. Get Scorebar (Live Games & Schedule)

The `get_scorebar()` function shows current/recent/upcoming games.

In [ ]:
# Get scorebar for PWHL (shows games from 3 days back to 7 days ahead)
scorebar = pwhl_api.get_scorebar(days_ahead=7, days_back=3)

print("PWHL Scorebar:")
print(f"Type: {type(scorebar)}")
if isinstance(scorebar, dict):
    games = scorebar.get('games', scorebar.get('SiteKit', {}).get('Scorebar', []))
    if isinstance(games, list):
        print(f"Found {len(games)} games")
        if games:
            print(f"\nSample game:")
            print(json.dumps(games[0], indent=2)[:500])
    else:
        print("No games found in expected format")
else:
    print(f"Retrieved data: {str(scorebar)[:200]}")

## 3. Player Statistics with Pagination

Get skater and goalie stats with filtering and pagination.

In [ ]:
# Get top 20 PWHL skaters
print("="*60)
print("TOP 20 PWHL SKATERS")
print("="*60)

skaters = pwhl_api.get_skater_stats(limit=20, first=0)

if isinstance(skaters, dict):
    player_list = skaters.get('players', skaters.get('SiteKit', {}).get('Player', []))
    
    if isinstance(player_list, list) and player_list:
        # Convert to DataFrame for better display
        df = pd.DataFrame(player_list)
        
        # Select interesting columns if they exist
        display_cols = []
        for col in ['name', 'team', 'position', 'games_played', 'goals', 'assists', 'points']:
            if col in df.columns:
                display_cols.append(col)
        
        if display_cols:
            print(df[display_cols].head(10))
        else:
            # Show all columns if we don't find the expected ones
            print(df.head(10))
    else:
        print("No player data found")

## 4. Team-Specific Statistics

Get skater stats filtered by a specific team.

In [ ]:
# Get AHL skaters for a specific team (team ID 440 = Abbotsford Canucks)
team_stats = ahl_api.get_skater_stats(team='440', limit=50)

print("="*60)
print("ABBOTSFORD CANUCKS SKATERS (AHL)")
print("="*60)

if isinstance(team_stats, dict):
    players = team_stats.get('players', team_stats.get('SiteKit', {}).get('Player', []))
    
    if isinstance(players, list):
        print(f"Found {len(players)} players on the roster")
        
        # Show top 5
        for i, player in enumerate(players[:5], 1):
            if isinstance(player, dict):
                name = player.get('name', 'Unknown')
                pos = player.get('position', '?')
                gp = player.get('games_played', 0)
                pts = player.get('points', 0)
                print(f"{i}. {name} ({pos}) - GP: {gp}, Pts: {pts}")

## 5. Goalie Statistics with Qualification Filter

Get qualified goalies only (minimum games played requirements).

In [ ]:
# Get OHL qualified goalies
print("="*60)
print("OHL QUALIFIED GOALIES")
print("="*60)

goalies = ohl_api.get_goalie_stats(qualified='qualified', limit=15)

if isinstance(goalies, dict):
    goalie_list = goalies.get('goalies', goalies.get('SiteKit', {}).get('Goalie', []))
    
    if isinstance(goalie_list, list) and goalie_list:
        df = pd.DataFrame(goalie_list)
        
        # Try to display relevant goalie stats
        display_cols = []
        for col in ['name', 'team', 'games_played', 'wins', 'losses', 'save_pct', 'gaa']:
            if col in df.columns:
                display_cols.append(col)
        
        if display_cols:
            print(df[display_cols].head(10))
        else:
            print(f"Columns available: {list(df.columns)[:10]}")
            print(df.head(5))
    else:
        print("No qualified goalies found")

## 6. Standings with Grouping Options

Get league standings grouped by division, conference, or league-wide.

In [ ]:
# Get QMJHL standings grouped by division
print("="*60)
print("QMJHL STANDINGS (by division)")
print("="*60)

standings = qmjhl_api.get_standings(
    context='overall',  # 'overall', 'home', or 'away'
    group_by='division'  # 'division', 'conference', or 'league'
)

if standings:
    print(f"Retrieved standings data")
    print(f"Type: {type(standings)}")
    
    # Try to extract teams
    if isinstance(standings, dict):
        teams = standings.get('teams', standings.get('SiteKit', {}).get('Team', []))
        if isinstance(teams, list):
            print(f"Found {len(teams)} teams in standings")

## 7. Full Season Schedule

Get the complete schedule for a season, optionally filtered by team.

In [ ]:
# Get WHL schedule for all teams
print("="*60)
print("WHL FULL SCHEDULE")
print("="*60)

schedule = whl_api.get_schedule(
    team_id=-1,  # -1 for all teams
    month=-1,  # -1 for all months, 1-12 for specific month
    location='homeaway'  # 'homeaway', 'home', or 'away'
)

if isinstance(schedule, dict):
    games = schedule.get('games', schedule.get('SiteKit', {}).get('Game', []))
    if isinstance(games, list):
        print(f"Found {len(games)} games in schedule")
        
        # Show first 3 games
        for i, game in enumerate(games[:3], 1):
            if isinstance(game, dict):
                home = game.get('home_team', 'Unknown')
                away = game.get('visitor_team', 'Unknown')
                date = game.get('date_played', 'Unknown')
                print(f"{i}. {away} @ {home} - {date}")

## 8. Rookie-Only Statistics

Filter player stats to show only rookies.

In [ ]:
# Get OHL rookies only
print("="*60)
print("OHL ROOKIE SKATERS")
print("="*60)

rookies = ohl_api.get_skater_stats(
    rookies=1,  # 1 for rookies only, 0 for all players
    limit=20
)

if isinstance(rookies, dict):
    rookie_list = rookies.get('players', rookies.get('SiteKit', {}).get('Player', []))
    
    if isinstance(rookie_list, list):
        print(f"Found {len(rookie_list)} rookie skaters")
        
        # Show top 5 rookies
        for i, player in enumerate(rookie_list[:5], 1):
            if isinstance(player, dict):
                name = player.get('name', 'Unknown')
                team = player.get('team', '?')
                pts = player.get('points', 0)
                print(f"{i}. {name} ({team}) - {pts} pts")

## 9. Team Roster with Status Filtering

Get a team's roster, optionally filtering by active/inactive status.

In [ ]:
# Get PWHL roster for a team
print("="*60)
print("PWHL TEAM ROSTER")
print("="*60)

# Note: Replace with actual team_id from get_teams()
roster = pwhl_api.get_roster(
    team_id=2,  # Example team ID
    roster_status='undefined'  # 'undefined' for all, 'active', 'inactive'
)

if roster:
    print(f"Retrieved roster data")
    print(f"Type: {type(roster)}")
    
    if isinstance(roster, dict):
        players = roster.get('players', roster.get('SiteKit', {}).get('Player', []))
        if isinstance(players, list):
            print(f"Roster size: {len(players)} players")

## 10. Game-Specific Data

Get detailed information about a specific game including preview, summary, and play-by-play.

In [ ]:
# Example: Get game preview for AHL
# Note: Use an actual game_id from get_scorebar() or get_schedule()

# Uncomment and replace with real game_id:
# game_id = 1028297
# 
# print("="*60)
# print(f"GAME {game_id} DATA")
# print("="*60)
# 
# # Get game preview
# preview = ahl_api.get_game_preview(game_id)
# print(f"Preview: {type(preview)}")
# 
# # Get game summary
# summary = ahl_api.get_game_summary(game_id)
# print(f"Summary: {type(summary)}")
# 
# # Get play-by-play
# pbp = ahl_api.get_play_by_play(game_id)
# print(f"Play-by-play: {type(pbp)}")
# if isinstance(pbp, list):
#     print(f"Found {len(pbp)} events")

print("Game-specific functions demonstrated (commented out - requires valid game_id)")

## 11. Convenience Functions

Each API module includes convenience functions for common tasks.

In [ ]:
# Get all players (skaters + goalies) at once
print("="*60)
print("ALL PLAYERS CONVENIENCE FUNCTION")
print("="*60)

all_players = pwhl_api.get_all_players()

print(f"Type: {type(all_players)}")
if isinstance(all_players, dict):
    skaters = all_players.get('skaters', [])
    goalies = all_players.get('goalies', [])
    print(f"Skaters: {len(skaters) if isinstance(skaters, list) else 'N/A'}")
    print(f"Goalies: {len(goalies) if isinstance(goalies, list) else 'N/A'}")

## 12. Bootstrap/Configuration Data

Get league configuration and metadata for initializing applications.

In [ ]:
# Get bootstrap configuration
print("="*60)
print("BOOTSTRAP CONFIGURATION")
print("="*60)

bootstrap = pwhl_api.get_bootstrap(
    season='latest',
    page_name='scorebar'
)

if bootstrap:
    print(f"Retrieved bootstrap data")
    print(f"Type: {type(bootstrap)}")
    
    if isinstance(bootstrap, dict):
        print(f"Keys: {list(bootstrap.keys())[:10]}")

## 13. Pagination Example

Demonstrate how to paginate through large datasets.

In [ ]:
# Paginate through all AHL skaters (100 per page)
print("="*60)
print("PAGINATION EXAMPLE")
print("="*60)

all_skaters = []
page = 0
page_size = 100
max_pages = 3  # Limit for demo

print(f"Fetching AHL skaters ({page_size} per page)...")

while page < max_pages:
    stats = ahl_api.get_skater_stats(first=page*page_size, limit=page_size)
    
    if isinstance(stats, dict):
        players = stats.get('players', stats.get('SiteKit', {}).get('Player', []))
        if isinstance(players, list) and players:
            all_skaters.extend(players)
            print(f"  Page {page+1}: {len(players)} players (total: {len(all_skaters)})")
            page += 1
        else:
            print(f"  No more players found")
            break
    else:
        print(f"  Unexpected response format")
        break

print(f"\nTotal skaters collected: {len(all_skaters)}")

## 14. Comparing Across Leagues

Since all APIs have the same structure, it's easy to compare data across leagues.

In [ ]:
# Compare number of teams across all leagues
print("="*60)
print("LEAGUE COMPARISON")
print("="*60)

comparison = []

for league_name, api_module in leagues:
    teams = api_module.get_teams()
    
    if isinstance(teams, dict):
        team_list = teams.get('teams', teams.get('SiteKit', {}).get('Team', []))
        if isinstance(team_list, list):
            comparison.append({
                'League': league_name,
                'Teams': len(team_list)
            })

# Create comparison DataFrame
df_comparison = pd.DataFrame(comparison)
print(df_comparison.to_string(index=False))

## 15. Custom Configuration

You can create custom configurations for any league.

In [ ]:
# Create custom PWHL configuration
custom_config = pwhl_api.PWHLConfig(
    default_season='2',  # Use a different default season
)

print("="*60)
print("CUSTOM CONFIGURATION")
print("="*60)
print(f"Custom config created with season: {custom_config.DEFAULT_SEASON}")
print(f"API Key: {custom_config.API_KEY[:10]}...")
print(f"Client Code: {custom_config.CLIENT_CODE}")

# Use custom config in API calls
# teams = pwhl_api.get_teams(config=custom_config)

## Summary

This notebook demonstrated:

1. ✅ **Basic API calls** - Getting teams, schedules, and games
2. ✅ **Player statistics** - Skaters and goalies with filtering
3. ✅ **Pagination** - Handling large datasets efficiently
4. ✅ **Filtering** - By team, rookies, qualified status
5. ✅ **Grouping** - Standings by division/conference
6. ✅ **Game data** - Preview, summary, play-by-play
7. ✅ **Convenience functions** - Simplified common tasks
8. ✅ **Custom configs** - Override default settings
9. ✅ **Cross-league** - Compare data across leagues

All 5 league APIs (PWHL, AHL, OHL, WHL, QMJHL) share the **same interface**, making it easy to:
- Switch between leagues by changing imports
- Build applications that support multiple leagues
- Reuse code across different leagues

### Key Features

- 🚀 **15+ endpoints** per league
- 🔄 **Pagination** support for large datasets
- 🎯 **Filtering** by team, division, conference, rookies, qualified
- ⚡ **Rate limiting** (2 requests/second) built-in
- 🛡️ **Error handling** with detailed logging
- 📝 **Type hints** for IDE autocomplete
- 🎨 **Consistent API** across all leagues